# <img src="https://files.training.databricks.com/images/DeltaLake-logo.png" width=80px> Open Source Delta Lake

[Delta Lake](https://delta.io/) is an open-source storage layer that brings ACID transactions to Apache Spark™ and big data workloads.

<img src="https://www.evernote.com/l/AAF4VIILJtFNZLuvZjGGhZTr2H6Z0wh6rOYB/image.png" width=900px>

### Key Features

[Quick start intro to Delta Lake.](https://docs.delta.io/latest/quick-start.html#)

**ACID Transactions**:
Data lakes typically have multiple data pipelines reading and writing data concurrently, and data engineers have to go through a tedious process to ensure data integrity, due to the lack of transactions. Delta Lake brings ACID transactions to your data lakes. It provides serializability, the strongest level of isolation level.

**Scalable Metadata Handling**:
In big data, even the metadata itself can be "big data". Delta Lake treats metadata just like data, leveraging Spark's distributed processing power to handle all its metadata. As a result, Delta Lake can handle petabyte-scale tables with billions of partitions and files at ease.

**Time Travel (data versioning)**:
Delta Lake provides snapshots of data enabling developers to access and revert to earlier versions of data for audits, rollbacks or to reproduce experiments.

**Open Format**:
All data in Delta Lake is stored in Apache Parquet format enabling Delta Lake to leverage the efficient compression and encoding schemes that are native to Parquet.

**Unified Batch and Streaming Source and Sink**:
A table in Delta Lake is both a batch table, as well as a streaming source and sink. Streaming data ingest, batch historic backfill, and interactive queries all just work out of the box.

**Schema Enforcement**:
Delta Lake provides the ability to specify your schema and enforce it. This helps ensure that the data types are correct and required columns are present, preventing bad data from causing data corruption.

**Schema Evolution**:
Big data is continuously changing. Delta Lake enables you to make changes to a table schema that can be applied automatically, without the need for cumbersome DDL.

**100% Compatible with Apache Spark API**:
Developers can use Delta Lake with their existing data pipelines with minimal change as it is fully compatible with Spark, the commonly used big data processing engine.

### Getting Started

You will notice that throughout this course, there is a lot of context switching between PySpark/Scala and SQL.

This is because:
* `read` and `write` operations are performed on DataFrames using PySpark or Scala
* table creates and queries are performed directly off Delta Lake tables using SQL

Run the following cell to configure our "classroom."

In [4]:
%run "./Includes/Classroom-Setup"

-sandbox
<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Key Concepts: Delta Lake Architecture</h2>

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> We'll touch on this further in future notebooks.

Throughout our Delta Lake discussions, we'll often refer to the concept of Bronze/Silver/Gold tables. These levels refer to the state of data refinement as data flows through a processing pipeline.

**These levels are conceptual guidelines, and implemented architectures may have any number of layers with various levels of enrichment.** Below are some general ideas about the state of data in each level.

* **Bronze** tables
  * Raw data (or very little processing)
  * Data will be stored in the Delta format (can encode raw bytes as a column)
* **Silver** tables
  * Data that is directly queryable and ready for insights
  * Bad records have been handled, types have been enforced
* **Gold** tables
  * Highly refined views of the data
  * Aggregate tables for BI
  * Feature tables for data scientists

For different workflows, things like schema enforcement and deduplication may happen in different places.

## Delta Lake Batch Operations - Create

Creating Delta Lakes is as easy as changing the file type while performing a write. 

In this section, we'll read from a CSV and write to Delta.

![](https://files.training.databricks.com/images/adbcore/AAFxQkg_SzRC06GvVeatDBnNbDL7wUUgCg4B.png)

Set up relevant paths to the online retail datasets from `/mnt/training/online_retail`

In [9]:
inputPath = "/mnt/training/online_retail/data-001/data.csv"
DataPath = userhome + "/delta/customer-data/"

#remove directory if it exists
dbutils.fs.rm(DataPath, True)

Out[5]: False

Read the data into a DataFrame. We supply the schema.

Use overwrite mode so that there will not be an issue in rewriting the data in case you end up running the cell again.

Partition on `Country` because there are only a few unique countries and because we will use `Country` as a predicate in a `WHERE` clause.

More information on the how and why of partitioning is contained in the links at the bottom of this notebook.

Then write the data to Delta Lake.

In [11]:
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

inputSchema = StructType([
  StructField("InvoiceNo", IntegerType(), True),
  StructField("StockCode", StringType(), True),
  StructField("Description", StringType(), True),
  StructField("Quantity", IntegerType(), True),
  StructField("InvoiceDate", StringType(), True),
  StructField("UnitPrice", DoubleType(), True),
  StructField("CustomerID", IntegerType(), True),
  StructField("Country", StringType(), True)
])

rawDataDF = (spark.read
  .option("header", "true")
  .schema(inputSchema)
  .csv(inputPath)
)

# write to Delta Lake
rawDataDF.write.mode("overwrite").format("delta").partitionBy("Country").save(DataPath)

-sandbox

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> While we show creating a table in the next section, Spark SQL queries can run directly on a directory of data, for delta use the following syntax: 
```
SELECT * FROM delta.`/path/to/delta_directory`
```

In [13]:
display(spark.sql("SELECT * FROM delta.`{}` LIMIT 5".format(DataPath)))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536389,22941,CHRISTMAS LIGHTS 10 REINDEER,6,12/1/10 10:03,8.5,12431,Australia
536389,21622,VINTAGE UNION JACK CUSHION COVER,8,12/1/10 10:03,4.95,12431,Australia
536389,21791,VINTAGE HEADS AND TAILS CARD GAME,12,12/1/10 10:03,1.25,12431,Australia
536389,35004C,SET OF 3 COLOURED FLYING DUCKS,6,12/1/10 10:03,5.45,12431,Australia
536389,35004G,SET OF 3 GOLD FLYING DUCKS,4,12/1/10 10:03,6.35,12431,Australia


-sandbox
### CREATE A Table Using Delta Lake

Create a table called `customer_data_delta` using `DELTA` out of the above data.

The notation is:
> `CREATE TABLE <table-name>` <br>
  `USING DELTA` <br>
  `LOCATION <path-do-data> ` <br>
  
Tables created with a specified `LOCATION` are considered unmanaged by the metastore. Unlike a managed table, where no path is specified, an unmanaged table’s files are not deleted when you `DROP` the table. However, changes to either the registered table or the files will be reflected in both locations.

<img alt="Best Practice" title="Best Practice" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-blue-ribbon.svg"/> Managed tables require that the data for your table be stored in DBFS. Unmanaged tables only store metadata in DBFS. 

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Since Delta Lake stores schema (and partition) info in the `_delta_log` directory, we do not have to specify partition columns!

In [15]:
spark.sql("""
  DROP TABLE IF EXISTS customer_data_delta
""")
spark.sql("""
  CREATE TABLE customer_data_delta
  USING DELTA
  LOCATION '{}'
""".format(DataPath))

Out[8]: DataFrame[]

-sandbox
Perform a simple `count` query to verify the number of records.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> Notice how the count is right off the bat; no need to worry about table repairs.

In [17]:
%sql
SELECT count(*) FROM customer_data_delta

count(1)
65499


### Metadata

Since we already have data backing `customer_data_delta` in place,
the table in the Hive metastore automatically inherits the schema, partitioning,
and table properties of the existing data.

Note that we only store table name, path, database info in the Hive metastore,
the actual schema is stored in the `_delta_log` directory as shown below.

In [19]:
display(dbutils.fs.ls(DataPath + "/_delta_log"))

path,name,size
dbfs:/user/damian.joseph@team.telstra.com/delta/customer-data/_delta_log/00000000000000000000.crc,00000000000000000000.crc,92
dbfs:/user/damian.joseph@team.telstra.com/delta/customer-data/_delta_log/00000000000000000000.json,00000000000000000000.json,29803
dbfs:/user/damian.joseph@team.telstra.com/delta/customer-data/_delta_log/__tmp_path_dir/,__tmp_path_dir/,0


Metadata is displayed through `DESCRIBE DETAIL <tableName>`.

As long as we have some data in place already for a Delta Lake table, we can infer schema.

In [21]:
%sql
DESCRIBE DETAIL customer_data_delta

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,db540b9d-ece7-49ba-b636-130b37c085a6,damian_joseph_team_telstra_com_db.customer_data_delta,null,dbfs:/user/damian.joseph@team.telstra.com/delta/customer-data,2021-05-31T15:36:29.291+0000,2021-05-31T15:36:39.000+0000,List(Country),37,634986,Map(),1,2


### Key Takeaways

Saving to Delta Lake is as easy as saving to Parquet, but creates an additional log file.

Using Delta Lake to create tables is straightforward and you do not need to specify schemas.

## Delta Lake Batch Operations - Append

In this section, we'll load a small amount of new data and show how easy it is to append this to our existing Delta table.

We'll start start by setting up our relevant path and loading new consumer product data.

In [24]:
miniDataInputPath = "/mnt/training/online_retail/outdoor-products/outdoor-products-mini.csv"

newDataDF = (spark
  .read
  .option("header", "true")
  .schema(inputSchema)
  .csv(miniDataInputPath)
)

Do a simple count of number of new items to be added to production data.

In [26]:
newDataDF.count()

Out[11]: 36

### APPEND Using Delta Lake

Adding to our existing Delta Lake is as easy as modifying our write statement and specifying the `append` mode. 

Here we save to our previously created Delta Lake at `delta/customer-data/`.

In [28]:
(newDataDF
  .write
  .format("delta")
  .partitionBy("Country")
  .mode("append")
  .save(DataPath)
)

-sandbox
Perform a simple `count` query to verify the number of records and notice it is correct.

Should be `65535`.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> The changes to our files have been immediately reflected in the table that we've registered.

In [30]:
%sql
SELECT count(*) FROM customer_data_delta

count(1)
65535


### Key Takeaways
With Delta Lake, you can easily append new data without schema-on-read issues.

Changes to Delta Lake files will immediately be reflected in registered Delta tables.

## Delta Lake Batch Operations - Upsert

To UPSERT means to "UPdate" and "inSERT". In other words, UPSERT is literally TWO operations. It is not supported in traditional data lakes, as running an UPDATE could invalidate data that is accessed by the subsequent INSERT operation.

Using Delta Lake, however, we can do UPSERTS. Delta Lake combines these operations to guarantee atomicity to
- INSERT a row 
- if the row already exists, UPDATE the row.

### Scenario
You have a small amount of batch data to write to your Delta table. This is currently staged in a JSON in a mounted blob store.

In [33]:
upsertDF = spark.read.format("json").load("/mnt/training/enb/commonfiles/upsert-data.json")
display(upsertDF)

Country,CustomerID,Description,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
Iceland,20993,Canyon Mule Journey Backpack,1/7/18 9:01,541699,132,39045,348.61
Iceland,20993,Canyon Mule Extreme Backpack,1/7/18 9:01,541699,124,31690,432.89
Iceland,20993,Canyon Mule Cooler,1/7/18 9:01,541699,382,31786,32.92
Iceland,20993,Firefly Lite,1/7/18 9:01,541699,705,37024,14.47
Iceland,20993,Firefly Mapreader,1/7/18 9:01,541699,455,39835,15.96
Iceland,20993,Firefly 2,1/7/18 9:01,541699,218,38474,26.82
Iceland,20993,Firefly 4,1/7/18 9:01,541699,595,38636,26.56966387
Iceland,20993,EverGlow Double,1/7/18 9:01,541699,22,34059,52.15
Iceland,20993,EverGlow Kerosene,1/7/18 9:01,541699,323,38732,30.92
Iceland,20993,EverGlow Lamp,1/7/18 9:01,541699,340,31822,27.25


We'll register this as a temporary view so that this table doesn't persist in DBFS (but we can still use SQL to query it).

In [35]:
upsertDF.createOrReplaceTempView("upsert_data")

Included in this data are:
- Some new orders for customer 20993
- An update to a previous order correcting the country for customer 20993 to Iceland
- Corrections to some records for StockCode 22837 where the Description was incorrect

We can use UPSERT to simultaneously INSERT our new data and UPDATE our previous records.

In [37]:
%sql
MERGE INTO customer_data_delta
USING upsert_data
ON customer_data_delta.InvoiceNo = upsert_data.InvoiceNo
  AND customer_data_delta.StockCode = upsert_data.StockCode
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED
  THEN INSERT *

Notice how this data is seamlessly incorporated into `customer_data_delta`.

In [39]:
%sql
SELECT * FROM customer_data_delta WHERE CustomerID=20993

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541699,34059,EverGlow Double,22,1/7/18 9:01,52.15,20993,Iceland
541699,38636,Firefly 4,595,1/7/18 9:01,26.56966387,20993,Iceland
541699,38732,EverGlow Kerosene,323,1/7/18 9:01,30.92,20993,Iceland
541699,39045,Canyon Mule Journey Backpack,132,1/7/18 9:01,348.61,20993,Iceland
536371,32129,EverGlow Single,228,1/1/18 9:01,33.85,20993,Iceland
541699,31786,Canyon Mule Cooler,382,1/7/18 9:01,32.92,20993,Iceland
541699,31822,EverGlow Lamp,340,1/7/18 9:01,27.25,20993,Iceland
541699,37024,Firefly Lite,705,1/7/18 9:01,14.47,20993,Iceland
541699,38474,Firefly 2,218,1/7/18 9:01,26.82,20993,Iceland
541699,31690,Canyon Mule Extreme Backpack,124,1/7/18 9:01,432.89,20993,Iceland


In [40]:
%sql
SELECT DISTINCT(Description) 
FROM customer_data_delta 
WHERE StockCode = 22837

Description
null
HOT WATER BOTTLE BABUSHKA


## Summary
In this Lesson, we:
- Saved files using Delta Lake
- Used Delta Lake to UPSERT data into existing Delta Lake tables

## Additional Topics & Resources

* <a href="https://docs.databricks.com/delta/delta-batch.html#" target="_blank">Table Batch Read and Writes</a>